In [1]:
import cv2
import numpy as np
import os
from itertools import permutations

EDGE_THICKNESS = 3
ERROR_LIMIT = 300
SAVE_DIR = "./results/2x2_out"


# ---------------- IMAGE SLICING ----------------
def slice_image_quadrants(img):
    h, w = img.shape[:2]
    h2, w2 = h // 2, w // 2
    return [
        img[:h2, :w2],
        img[:h2, w2:],
        img[h2:, :w2],
        img[h2:, w2:]
    ]


# ---------------- EDGE HANDLING ----------------
def extract_edge_strip(img, side):
    t = EDGE_THICKNESS
    return {
        'top': img[:t],
        'bottom': img[-t:],
        'left': img[:, :t],
        'right': img[:, -t:]
    }[side]


def compute_edge_descriptor(edge):
    edge = cv2.GaussianBlur(edge, (3,3), 0)

    lab  = cv2.cvtColor(edge, cv2.COLOR_BGR2LAB).astype(np.float32)
    gray = cv2.cvtColor(edge, cv2.COLOR_BGR2GRAY).astype(np.float32)

    gx = cv2.Sobel(gray, cv2.CV_32F, 1, 0)
    gy = cv2.Sobel(gray, cv2.CV_32F, 0, 1)

    grad = np.sqrt(gx**2 + gy**2)[...,None]
    lap  = cv2.Laplacian(gray, cv2.CV_32F)[...,None]

    return np.concatenate([lab, grad, lap], axis=2)


def standardize_features(f):
    f = f.copy()
    for c in range(f.shape[2]):
        mu, sigma = f[...,c].mean(), f[...,c].std()
        f[...,c] = (f[...,c] - mu) / (sigma if sigma > 1e-6 else 1)
    return f


def compare_edge_similarity(a, b, sa, sb):
    a = standardize_features(a)
    b = standardize_features(b)

    if sa in ('left','right'):
        a = np.transpose(a, (1,0,2))
    if sb in ('left','right'):
        b = np.transpose(b, (1,0,2))

    if a.shape != b.shape:
        b = cv2.resize(b, (a.shape[1], a.shape[0]))

    diff = np.abs(a - b)

    return (
        0.5 * diff[...,0:3].mean() +
        0.3 * diff[...,3].mean() +
        0.2 * diff[...,4].mean()
    )


# ---------------- OPTIMIZATION ----------------
def generate_edge_costs(pieces):
    sides = ['top','bottom','left','right']
    opposite = {'top':'bottom','bottom':'top','left':'right','right':'left'}
    n = len(pieces)

    descriptors = {
        (i,s): compute_edge_descriptor(extract_edge_strip(p,s))
        for i,p in enumerate(pieces)
        for s in sides
    }

    costs = {s: np.full((n,n), np.inf) for s in sides}

    for i in range(n):
        for j in range(n):
            if i == j:
                continue
            for s in sides:
                costs[s][i,j] = compare_edge_similarity(
                    descriptors[(i,s)],
                    descriptors[(j,opposite[s])],
                    s, opposite[s]
                )
    return costs


def optimize_2x2_layout(pieces):
    cost = generate_edge_costs(pieces)
    best, runner_up = None, 1e18
    best_score = 1e18

    for p in permutations(range(4)):
        score = (
            cost['right'][p[0], p[1]] +
            cost['bottom'][p[0], p[2]] +
            cost['right'][p[2], p[3]] +
            cost['bottom'][p[1], p[3]]
        )
        if score < best_score:
            runner_up = best_score
            best_score = score
            best = p
        elif score < runner_up:
            runner_up = score

    return list(best), runner_up - best_score


# ---------------- RECONSTRUCTION ----------------
def reconstruct_image(pieces, order):
    h, w = pieces[0].shape[:2]
    canvas = np.zeros((2*h,2*w,3), np.uint8)

    canvas[:h,:w]     = pieces[order[0]]
    canvas[:h,w:]     = pieces[order[1]]
    canvas[h:,:w]     = pieces[order[2]]
    canvas[h:,w:]     = pieces[order[3]]

    return canvas


def mean_squared_error(a, b):
    if a.shape != b.shape:
        b = cv2.resize(b, (a.shape[1], a.shape[0]))
    return np.mean((a.astype(np.float32) - b.astype(np.float32))**2)


# ---------------- PIPELINE ----------------
def run_solver_and_evaluate(puzzle_dir, gt_dir):
    os.makedirs(SAVE_DIR, exist_ok=True)

    files = sorted(
        [f for f in os.listdir(puzzle_dir) if f.lower().endswith(('.png','.jpg'))],
        key=lambda x: int(''.join(c for c in x if c.isdigit()) or 0)
    )

    correct = total = 0

    print("\n[INFO] Edge-driven 2x2 Puzzle Reconstruction")
    print("-"*60)

    for idx, fname in enumerate(files, 1):
        img = cv2.imread(os.path.join(puzzle_dir, fname))
        if img is None:
            continue

        pieces = slice_image_quadrants(img)
        order, confidence = optimize_2x2_layout(pieces)
        result = reconstruct_image(pieces, order)

        pid = os.path.splitext(fname)[0]
        cv2.imwrite(os.path.join(SAVE_DIR, pid+".png"), result)

        gt = cv2.imread(os.path.join(gt_dir, pid+".png"))
        if gt is None:
            continue

        err = mean_squared_error(result, gt)
        passed = err < ERROR_LIMIT

        correct += passed
        total += 1

        print(f"{idx:3d} | {pid:<6} | error={err:7.1f} | "
              f"{'OK' if passed else 'FAIL'} | confidence={confidence:.2f}")

    print("-"*60)
    print(f"Accuracy: {correct}/{total} = {100*correct/total:.2f}%")
    print(f"Saved results in: {SAVE_DIR}\n")


if __name__ == "__main__":
    run_solver_and_evaluate(
        r"C:\Term 5\image_project\OneDrive_2025-11-26\Jigsaw Puzzle Dataset\Gravity Falls\puzzle_2x2",
        r"C:\Term 5\image_project\OneDrive_2025-11-26\Jigsaw Puzzle Dataset\Gravity Falls\correct"
    )



[INFO] Edge-driven 2x2 Puzzle Reconstruction
------------------------------------------------------------
  1 | 0      | error=   64.9 | OK | confidence=0.47
  2 | 1      | error=   15.7 | OK | confidence=0.59
  3 | 2      | error=   32.4 | OK | confidence=0.57
  4 | 3      | error= 9669.3 | FAIL | confidence=0.11
  5 | 4      | error= 9586.5 | FAIL | confidence=0.76
  6 | 5      | error= 9093.2 | FAIL | confidence=0.92
  7 | 6      | error= 9173.1 | FAIL | confidence=0.51
  8 | 7      | error=   11.5 | OK | confidence=0.63
  9 | 8      | error=   26.2 | OK | confidence=0.82
 10 | 9      | error=   29.6 | OK | confidence=0.67
 11 | 10     | error=   24.2 | OK | confidence=0.13
 12 | 11     | error=   29.6 | OK | confidence=0.71
 13 | 12     | error=   24.2 | OK | confidence=0.46
 14 | 13     | error=   38.2 | OK | confidence=0.47
 15 | 14     | error=   56.9 | OK | confidence=0.55
 16 | 15     | error=   64.1 | OK | confidence=0.68
 17 | 16     | error=   17.4 | OK | confidence=1.01
 